<a href="https://colab.research.google.com/github/youseefmoemen/Neural-Network/blob/main/Sup2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
def sigmoid(z):
  return 1 / (1 + np.exp(-1 * z))
def sigmoid_grad(z):
  t = sigmoid(z)
  return t * (1 - t)
def leakyRelu(z):
  return np.where(z >= 0, z, z * 0.3)
def leakyRelu_grad(z):
  return np.where(z >= 0, 1, 0.3)
def tanh(z):
  return np.tanh(z)
def tanh_grad(z):
  return 1 - np.power(tanh(z), 2)

In [3]:
Activations = {
    'sigmoid': sigmoid,
    'leakyRelu': leakyRelu,
    'softmax': softmax,
    'tanh': tanh
}
Activations_grad = {
    'sigmoid': sigmoid_grad,
    'leakyRelu': leakyRelu_grad,
    'softmax': softmax_grad,
    'tanh': tanh_grad
}

In [4]:
from sklearn.datasets import fetch_openml
import math

In [8]:
mnist = fetch_openml('mnist_784', version = 1)

In [9]:
from sklearn.model_selection import StratifiedShuffleSplit

In [10]:
sss = StratifiedShuffleSplit(n_splits = 1, train_size=50000, test_size=10000)

In [133]:
for train_index, test_index in sss.split(mnist.data, mnist.target):
  X_train = mnist.data.iloc[train_index] 
  y_train = mnist.target[train_index]
  X_test = mnist.data.iloc[test_index] 
  y_test = mnist.target[test_index]

In [49]:
def clip_image(img, clipping_size = 0):
  img = img.reshape((28, 28))
  img = np.delete(img, range(clipping_size), 0)
  img = np.delete(img, range(clipping_size), 1)
  return img.flatten()

In [50]:
def momentum(img, step_size, n_steps, clipping_size):
  img = img.reshape((28 - clipping_size, 28-clipping_size))
  momentums = []
  for i in range(n_steps):
    x = np.array(list(range(i*step_size, (i+1)*step_size)))
    for j in range(n_steps):
      pixels = img[j*step_size: (j+1)*step_size, i*step_size: (i+1) * step_size]
      y =  np.array(list(range(j*step_size, (j+1)*step_size))).reshape(step_size,1)
      area = np.sum(np.sum(pixels))
      x_c = np.sum(np.sum(x.T * pixels)) / (area + 1e-10)
      y_c = np.sum(np.sum(y * pixels)) / (area + 1e-10)
      momentums.append((x_c, y_c))
  return momentums

In [51]:
def reduction(data = None, n_momentums = 9):
  n_steps = math.floor(np.sqrt(n_momentums))
  clipping_size = 28 % n_steps
  step_size = (28 - clipping_size) // n_steps
  data = np.apply_along_axis(clipping_size = clipping_size
                      , func1d = clip_image, axis = 1, arr = data)
  momentums = np.apply_along_axis(step_size = step_size, n_steps = n_steps,
                                  clipping_size= clipping_size,
                                  func1d = momentum, axis = 1, arr = data)
  return data, momentums

In [134]:
X_train = reduction(np.array(X_train.copy()), n_momentums = 9)[1].reshape(50000, 18)
y_train2 = np.zeros(shape=(y_train.shape[0], 10))
for index, instance in enumerate(y_train):
  y_train2[index][int(instance)] = 1
y_train = y_train2

In [94]:
class Layer():
  def __init__(self, n_neurons, activation):
    self.n_neurons = n_neurons
    self.w = None
    self.activation = Activations[activation]
    self.activation_grad = Activations_grad[activation]
  
  def initialize(self, prev):
    self.w = np.random.normal(size=(self.n_neurons, prev))

  def compute(self, Oi):
    Netj = np.dot(Oi, self.w.T)  
    Oj = self.activation(Netj)
    return Netj, Oj
  
  def grad(self, delta, w_next, Oi):
    Netj = self.compute(Oi)[0]
    partial = np.dot(delta, w_next)
    delta = np.multiply(partial, self.activation_grad(Netj))
    Dw = np.dot(delta.T, Oi)
    return Dw, delta

  def update(self, learning_rate, Dw):
    hold = self.w.shape
    self.w  = self.w  -  learning_rate * Dw

In [95]:
class OutputLayer(Layer):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
  
  def grad(self, target, Oi):
    Netj, Oj = self.compute(Oi)
    delta = np.multiply(-1* (1/target.shape[0]) * (target - Oj), self.activation_grad(Netj))
    Dw = np.dot(delta.T, Oi)
    return Dw, delta

In [113]:
class NeuralNetwork():
  def __init__(self, layers, input_shape):
    self.layers = layers
    self.O_ALL = None
    self.input_shape = input_shape
    self.layers[0].initialize(self.input_shape)
    for i in range(1, len(self.layers)):
      self.layers[i].initialize(self.layers[i-1].n_neurons)
  
  def predict(self, X):
    Oi = X
    self.O_ALL = []
    self.O_ALL.append(X)
    for layer in self.layers:
      Oi = layer.compute(Oi)[1]
      self.O_ALL.append(Oi)
    return Oi
  

  def loss(self, Oj, y):
    E = (2 / y.shape[0]) * np.sum((y - Oj) ** 2)
    return E
  
  def arg(self, y):
    predictions = np.zeros((y.shape[0], 1))
    for index, instance in enumerate(y):
      idx = 0
      for i in range(1, 10):
        if instance[i] > instance[idx]:
          idx = i
      predictions[index] = idx
    return predictions

  def accuracy(self, y_pred, y_true):
    return np.sum(np.equal(self.arg(y_true), self.arg(y_pred))) / len(y_true)

  def train(self, X_train, y_train, learning_rate, epochs, 
            X_val = None, y_val = None):
    for i in range(epochs):
        y_pred = self.predict(X_train)
        print(f'epoch {i} accuracy {self.accuracy(y_pred, y_train)}' ,
              f'loss {self.loss(y_train, y_pred)}')
        for index, instance in enumerate(X_train):
          self.predict(instance.reshape(1, -1))
          for j in reversed(range(len(self.layers))):
            if j == len(self.layers) - 1:
              Dw, delta = self.layers[j].grad(y_train[index], self.O_ALL[-2])
            else:
              Dw, delta = self.layers[j].grad(delta, hold, self.O_ALL[j])
            hold = self.layers[j].w.copy()
            self.layers[j].update(learning_rate, Dw)

In [135]:
layers2 = [
    Layer(10, 'sigmoid'),
    Layer(10, 'sigmoid'),
    OutputLayer(10, 'sigmoid')
]
nn2 = NeuralNetwork(layers2, 18)
nn2.train(X_train, y_train, 1e-7, 10)

epoch 0 accuracy 0.051 loss 10.606282083284972
epoch 1 accuracy 0.051 loss 10.604816549083935
epoch 2 accuracy 0.051 loss 10.603350501235964
epoch 3 accuracy 0.051 loss 10.601883946034476
epoch 4 accuracy 0.051 loss 10.600416889791443
epoch 5 accuracy 0.051 loss 10.59894933883722
epoch 6 accuracy 0.051 loss 10.597481299519922
epoch 7 accuracy 0.051 loss 10.596012778205125
epoch 8 accuracy 0.051 loss 10.594543781275409
epoch 9 accuracy 0.051 loss 10.593074315130183


In [84]:
layers = [
    Layer(35, 'leakyRelu'),
    Layer(20, 'leakyRelu'),
    OutputLayer(10, 'leakyRelu')
]
nn = NeuralNetwork(layers, 18)
nn.train(X_train, y_train, 1e-7, 1000)

iteration 0 accuracy 0.10706 loss 4973114.190725495
iteration 1 accuracy 0.0872 loss 16733.089653797753
iteration 2 accuracy 0.0827 loss 8950.565776341364
iteration 3 accuracy 0.0806 loss 6120.3890279291
iteration 4 accuracy 0.0804 loss 4767.611759023827
iteration 5 accuracy 0.0796 loss 3924.412014055008
iteration 6 accuracy 0.07954 loss 3333.8984557330405
iteration 7 accuracy 0.08008 loss 2894.9111716518937
iteration 8 accuracy 0.08018 loss 2556.3578980559487
iteration 9 accuracy 0.07998 loss 2288.2295193494356
iteration 10 accuracy 0.08116 loss 2069.6612726637954
iteration 11 accuracy 0.0822 loss 1888.7495025891533
iteration 12 accuracy 0.08366 loss 1736.1106994707134
iteration 13 accuracy 0.08444 loss 1605.008249212348
iteration 14 accuracy 0.08646 loss 1491.2001874831506
iteration 15 accuracy 0.0889 loss 1391.7717067544365
iteration 16 accuracy 0.09136 loss 1304.9180143227186
iteration 17 accuracy 0.09284 loss 1228.186795884842
iteration 18 accuracy 0.09426 loss 1159.7845231318906
